In [2]:
!pip install fastembed
!python -m pip install -q "qdrant-client[fastembed]>=1.14.2"

In [3]:
from fastembed import TextEmbedding
from qdrant_client import QdrantClient, models
import requests

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
for embedding in TextEmbedding.list_supported_models():
    if embedding['model'].startswith('jinaai'):
        print(embedding) 

# jinaai/jina-embeddings-v2-small-en 512

{'model': 'jinaai/jina-clip-v1', 'sources': {'hf': 'jinaai/jina-clip-v1', 'url': None, '_deprecated_tar_struct': False}, 'model_file': 'onnx/text_model.onnx', 'description': 'Text embeddings, Multimodal (text&image), English, Prefixes for queries/documents: not necessary, 2024 year', 'license': 'apache-2.0', 'size_in_GB': 0.55, 'additional_files': [], 'dim': 768, 'tasks': {}}
{'model': 'jinaai/jina-embeddings-v2-base-en', 'sources': {'hf': 'xenova/jina-embeddings-v2-base-en', 'url': None, '_deprecated_tar_struct': False}, 'model_file': 'onnx/model.onnx', 'description': 'Text embeddings, Unimodal (text), English, 8192 input tokens truncation, Prefixes for queries/documents: not necessary, 2023 year.', 'license': 'apache-2.0', 'size_in_GB': 0.52, 'additional_files': [], 'dim': 768, 'tasks': {}}
{'model': 'jinaai/jina-embeddings-v2-small-en', 'sources': {'hf': 'xenova/jina-embeddings-v2-small-en', 'url': None, '_deprecated_tar_struct': False}, 'model_file': 'onnx/model.onnx', 'description

In [22]:
model_handle = "jinaai/jina-embeddings-v2-small-en"
EMBEDDING_DIMENSIONALITY = 512

In [55]:
# Q1
from fastembed import TextEmbedding
import numpy as np

# Initialize the embedding model
embedding_model = TextEmbedding(model_name='jinaai/jina-embeddings-v2-small-en')

# Embed the query
query = 'I just discovered the course. Can I join now?'
query_embed = list(embedding_model.embed([query])) # embed returns a generator, convert to list
np.min(query_embed)

np.float64(-0.11726373885183883)

In [57]:
np.linalg.norm(query_embed)

np.float64(1.0)

In [58]:
query2 = 'Can I still join the course after the start date?'
query2_embed = list(embedding_model.embed([query2]))
query2_embed

[array([-5.45303910e-02, -7.83451754e-02,  3.13610346e-02,  2.34234938e-02,
        -3.06321480e-02, -5.51041640e-02,  2.24726486e-02,  1.50386159e-02,
        -1.81159589e-02,  1.80595354e-02, -4.80892277e-03, -7.57395955e-02,
        -4.87272162e-03, -9.07717788e-02,  4.83940879e-02, -4.26317430e-02,
        -2.61462523e-02, -4.48129135e-02, -2.06926015e-02, -3.79054485e-02,
        -3.30329342e-02, -8.02850120e-03,  1.18560351e-02, -3.83721199e-02,
         7.26378149e-02, -5.82811752e-02, -7.30890167e-02,  3.40911381e-02,
         5.60705346e-02,  5.02684759e-02, -5.99293341e-02,  5.33016566e-02,
        -1.79524520e-02,  4.17745107e-03, -2.45427902e-02,  1.32288255e-02,
         4.64178315e-02,  1.09963765e-02, -1.01499639e-02, -5.77524144e-03,
        -2.83987757e-04,  4.91496354e-02,  1.11214836e-01, -2.76306424e-02,
        -6.78923985e-02, -1.85928261e-02, -1.23963205e-01,  1.25745922e-02,
         1.65298714e-02,  7.22242999e-03, -2.77237445e-02,  5.18073540e-02,
        -5.9

In [65]:
query_embed[0]

array([-7.63946260e-02, -7.30555505e-02,  5.86501763e-02,  3.92670571e-02,
       -1.41797115e-02, -4.68342854e-02,  2.69862822e-02,  3.49039060e-02,
        1.53419217e-03, -1.10407363e-02,  5.57556285e-02, -2.88360142e-02,
       -4.87239498e-02, -9.90923923e-02,  6.02579764e-02, -1.86105379e-02,
       -8.81040139e-03, -3.01994831e-02, -1.42541815e-02, -3.87884212e-02,
       -2.76125662e-02, -6.94957533e-03,  3.20653319e-02, -5.17838394e-03,
        8.37959894e-02, -8.87329606e-02, -7.30260625e-02,  5.92846802e-02,
        4.07805674e-02,  7.72684822e-02, -4.60029697e-02,  3.98426895e-02,
        1.74821510e-02,  8.70263712e-03, -3.09291054e-02,  2.20470391e-02,
        4.76479896e-02,  1.76566196e-02, -3.62013627e-02, -3.53408062e-02,
       -5.92537995e-03,  2.97091202e-02,  8.05883752e-02,  1.07132722e-02,
       -5.38192519e-02, -9.62881111e-03, -1.17263739e-01,  3.36546374e-02,
        7.05763168e-03,  7.70159791e-03, -5.17482529e-02,  5.46955574e-02,
       -7.80289904e-02,  

In [ ]:
def cosine_similarity(vec1, vec2):
    vec1 = vec1.squeeze() # Ensure 1D
    vec2 = vec2.squeeze() # Ensure 1D

    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)

    if norm_vec1 == 0 or norm_vec2 == 0:
        return 0.0
    return dot_product / (norm_vec1 * norm_vec2)

In [67]:
# Q2
cosine_similarity(query_embed[0], query2_embed[0])

np.float64(0.9008528895674548)

In [68]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [69]:
# --- Query Embedding ---
query_vector = query_embed[0] # Shape (1, 512)

# --- Document Embeddings ---
document_texts = [doc['text'] for doc in documents]
document_embeddings_list = list(embedding_model.embed(document_texts))

# Convert the list of (1, 512) arrays to a single (num_documents, 512) matrix
# This is where the hint V.dot(q) comes into play if you wanted to use it.
# However, for clarity, we'll iterate and use our cosine_similarity function.
document_embedding_matrix = np.vstack([emb.squeeze() for emb in document_embeddings_list]) # Shape (5, 512)

# --- Calculate Cosine Similarities ---
similarities = []
for i, doc_vector_2d in enumerate(document_embeddings_list):
    # doc_vector_2d is still (1, 512), the cosine_similarity function handles squeezing
    sim = cosine_similarity(query_vector, doc_vector_2d)
    similarities.append(sim)
    print(f"Similarity with Document {i}: {sim:.4f}")

# Find the document index with the highest similarity
highest_similarity_index = np.argmax(similarities)
max_similarity_value = similarities[highest_similarity_index]

print(f"\nDocument index with the highest similarity: {highest_similarity_index}")
print(f"Highest similarity value: {max_similarity_value:.4f}")

Similarity with Document 0: 0.7630
Similarity with Document 1: 0.8182
Similarity with Document 2: 0.8085
Similarity with Document 3: 0.7133
Similarity with Document 4: 0.7304

Document index with the highest similarity: 1
Highest similarity value: 0.8182


In [70]:
# --- Query Embedding ---
query_vector = query_embed[0] # Shape (1, 512)

# --- Document Embeddings ---
document_texts = [doc['question'] + ' ' + doc['text'] for doc in documents]
document_embeddings_list = list(embedding_model.embed(document_texts))

# Convert the list of (1, 512) arrays to a single (num_documents, 512) matrix
# This is where the hint V.dot(q) comes into play if you wanted to use it.
# However, for clarity, we'll iterate and use our cosine_similarity function.
document_embedding_matrix = np.vstack([emb.squeeze() for emb in document_embeddings_list]) # Shape (5, 512)

# --- Calculate Cosine Similarities ---
similarities = []
for i, doc_vector_2d in enumerate(document_embeddings_list):
    # doc_vector_2d is still (1, 512), the cosine_similarity function handles squeezing
    sim = cosine_similarity(query_vector, doc_vector_2d)
    similarities.append(sim)
    print(f"Similarity with Document {i}: {sim:.4f}")

# Find the document index with the highest similarity
highest_similarity_index = np.argmax(similarities)
max_similarity_value = similarities[highest_similarity_index]

print(f"\nDocument index with the highest similarity: {highest_similarity_index}")
print(f"Highest similarity value: {max_similarity_value:.4f}")





Similarity with Document 0: 0.8515
Similarity with Document 1: 0.8437
Similarity with Document 2: 0.8408
Similarity with Document 3: 0.7755
Similarity with Document 4: 0.8086

Document index with the highest similarity: 0
Highest similarity value: 0.8515


In [73]:
# Q5
min_dim = min([embedding['dim'] for embedding in TextEmbedding.list_supported_models()])
min_dim

384

In [ ]:
for embedding in TextEmbedding.list_supported_models():
    if embedding['dim'] == min_dim:
        print(embedding)

# BAAI/bge-small-en

{'model': 'BAAI/bge-small-en', 'sources': {'hf': 'Qdrant/bge-small-en', 'url': 'https://storage.googleapis.com/qdrant-fastembed/BAAI-bge-small-en.tar.gz', '_deprecated_tar_struct': True}, 'model_file': 'model_optimized.onnx', 'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.', 'license': 'mit', 'size_in_GB': 0.13, 'additional_files': [], 'dim': 384, 'tasks': {}}
{'model': 'BAAI/bge-small-en-v1.5', 'sources': {'hf': 'qdrant/bge-small-en-v1.5-onnx-q', 'url': None, '_deprecated_tar_struct': False}, 'model_file': 'model_optimized.onnx', 'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.', 'license': 'mit', 'size_in_GB': 0.067, 'additional_files': [], 'dim': 384, 'tasks': {}}
{'model': 'snowflake/snowflake-arctic-embed-xs', 'sources': {'hf': 'snowflake/snowflake-arctic-embed-xs', 'url': None, '_deprecate

In [5]:
import requests 

client = QdrantClient("http://localhost:6333") 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)


model_handle = "BAAI/bge-small-en"
EMBEDDING_DIMENSIONALITY = 384

# Define the collection name
collection_name = "mlzoomcamp-rag"

# Create the collection with specified vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

points = []
id = 0


for doc in documents:

    point = models.PointStruct(
        id=id,
        vector=models.Document(text=doc['question']+' '+ doc['text'], model=model_handle),
        payload={
            "text": doc['text'],
            "section": doc['section'],
            "question": doc['question']
        } #save all needed metadata fields
    )
    points.append(point)

    id += 1

client.upsert(
collection_name=collection_name,
points=points
)



UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [6]:
def search(query, limit=1):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=query,
            model=model_handle 
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

question = 'I just discovered the course. Can I join now?'
result = search(question)
result

QueryResponse(points=[ScoredPoint(id=14, version=0, score=0.87031734, payload={'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'question': 'The course has already started. Can I still join it?'}, vector=None, shard_key=None, order_value=None)])